In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve, average_precision_score
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

2025-07-06 14:10:08.533524: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751811008.740448      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751811008.807027      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Load and clean data
df = pd.read_csv("/kaggle/input/mental-health-insights-data/Mental Health Dataset.csv")
if 'Unnamed: 0' in df.columns:
    df.drop('Unnamed: 0', axis=1, inplace=True)
df.dropna(subset=['posts', 'predicted'], inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

In [4]:
# Resample to balance classes 
max_count = df['predicted'].value_counts().max()
df_bal = pd.DataFrame()
for predicted, grp in df.groupby('predicted'):
    if len(grp) < max_count:
        grp_res = resample(grp, replace=True, n_samples=max_count, random_state=42)
    else:
        grp_res = grp
    df_bal = pd.concat([df_bal, grp_res])

df_bal.reset_index(drop=True, inplace=True)
print("After resampling:\n", df_bal['predicted'].value_counts())

df = df_bal


After resampling:
 predicted
negative         4374
neutral          4374
positive         4374
very negative    4374
Name: count, dtype: int64


In [5]:
# Build dynamic label map and one-hot labels
target_names = sorted(df['predicted'].unique())
num_classes = len(target_names)
label_map = {label: idx for idx, label in enumerate(target_names)}
df['label'] = df['predicted'].map(label_map)

In [6]:
# Tokenize with BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
MAX_LEN = 128
enc = tokenizer(
    df['posts'].astype(str).tolist(), padding='max_length', truncation=True,
    max_length=MAX_LEN, return_tensors='np'
)
input_ids = enc['input_ids']
attention_mask = enc['attention_mask']


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
# Prepare labels and tf.data dataset
labels = tf.keras.utils.to_categorical(df['label'], num_classes=num_classes)
dataset = tf.data.Dataset.from_tensor_slices(
    ({'input_ids': input_ids, 'attention_mask': attention_mask}, labels)
)

I0000 00:00:1751811257.333855      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1751811257.334517      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [8]:
# Train/val/test split
BATCH = 32
dataset = dataset.shuffle(buffer_size=len(df), seed=42)
n = len(df)
n_train = int(0.8 * n)
n_val = int(0.1 * n)
train_ds = dataset.take(n_train).batch(BATCH)
val_ds = dataset.skip(n_train).take(n_val).batch(BATCH)
test_ds = dataset.skip(n_train + n_val).batch(BATCH)


In [9]:
# Build model
def build_model():
    bert = TFBertModel.from_pretrained('bert-base-uncased')
    in_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_ids')
    in_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='attention_mask')
    x = bert(in_ids, attention_mask=in_mask).pooler_output
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    out = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    return tf.keras.Model(inputs=[in_ids, in_mask], outputs=out)

model = build_model()
model.summary()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',           
 )                           ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(None, 128, 7                                       

In [11]:
# Compile model
optim = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-8)
metrics = ['accuracy']
for cls, idx in label_map.items():
    metrics += [
        tf.keras.metrics.Precision(name=f"prec_{cls}", class_id=idx),
        tf.keras.metrics.Recall(name=f"rec_{cls}", class_id=idx)
    ]

model.compile(
    optimizer=optim,
    loss='categorical_crossentropy',
    metrics=metrics
)


In [12]:
# Train
epochs = 10
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]
hist = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/10


I0000 00:00:1751811320.501885     101 service.cc:148] XLA service 0x7846989880c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751811320.502509     101 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1751811320.502551     101 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1751811320.582575     101 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1751811320.710881     101 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


438/438 [==============================] - ETA: 0s - loss: 0.7461 - accuracy: 0.6986 - prec_negative: 0.6873 - rec_negative: 0.4947 - prec_neutral: 0.7806 - rec_neutral: 0.5991 - prec_positive: 0.7942 - rec_positive: 0.6354 - prec_very negative: 0.8327 - rec_very negative: 0.7501

/usr/local/lib/python3.11/dist-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


438/438 [==============================] - 439s 889ms/step - loss: 0.7461 - accuracy: 0.6986 - prec_negative: 0.6873 - rec_negative: 0.4947 - prec_neutral: 0.7806 - rec_neutral: 0.5991 - prec_positive: 0.7942 - rec_positive: 0.6354 - prec_very negative: 0.8327 - rec_very negative: 0.7501 - val_loss: 0.4550 - val_accuracy: 0.8130 - val_prec_negative: 0.6086 - val_rec_negative: 0.9320 - val_prec_neutral: 0.9635 - val_rec_neutral: 0.7237 - val_prec_positive: 0.9228 - val_rec_positive: 0.9651 - val_prec_very negative: 0.9923 - val_rec_very negative: 0.5828
Epoch 2/10
438/438 [==============================] - 390s 892ms/step - loss: 0.3305 - accuracy: 0.8860 - prec_negative: 0.8187 - rec_negative: 0.8010 - prec_neutral: 0.8822 - rec_neutral: 0.8162 - prec_positive: 0.9424 - rec_positive: 0.9437 - prec_very negative: 0.9339 - rec_very negative: 0.9399 - val_loss: 0.1619 - val_accuracy: 0.9531 - val_prec_negative: 0.9344 - val_rec_negative: 0.8946 - val_prec_neutral: 0.9017 - val_rec_neutral

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

In [14]:
from sklearn.model_selection import train_test_split

# Re-split df the same way for test set (same stratified logic)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
val_df, test_df   = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])


In [16]:
# Tokenize the test_df only
enc_test = tokenizer(
    test_df['posts'].astype(str).tolist(),
    padding='max_length',
    truncation=True,
    max_length=MAX_LEN,
    return_tensors='np'
)


In [17]:
# Run prediction using the in-memory trained model
test_inputs = {'input_ids': enc_test['input_ids'], 'attention_mask': enc_test['attention_mask']}
probs = model.predict(test_inputs, batch_size=BATCH)
y_pred_idx = probs.argmax(axis=1)


55/55 [==============================] - 20s 279ms/step


In [18]:
# Convert back to label names
inv_label_map = {v: k for k, v in label_map.items()}
bert_preds = [inv_label_map[i] for i in y_pred_idx]

In [20]:
# Create aligned DataFrame
df_bert = pd.DataFrame({
    'text': test_df['posts'].tolist(),
    'true_label': test_df['predicted'].tolist(),
    'bert_pred': bert_preds
})

In [21]:
# Save to CSV
df_bert.to_csv('predictions_bert.csv', index=False)
print("✅ predictions_bert.csv saved with", len(df_bert), "rows")
print(df_bert.head())


✅ predictions_bert.csv saved with 1750 rows
                                                text true_label bert_pred
0  hi all I have a 17yr old use weed to cope with...   negative  negative
1  5 1 cm x 5 8 cm x 5 6 cm irregular liver lesio...   negative  negative
2  I just want to let everyone know how I am do a...   positive  positive
3  have anyone ever hear of anyone have radiother...    neutral   neutral
4  ok I think this is by a long shot but really j...   negative  negative
